<a href="https://colab.research.google.com/github/haniehalipour/func-app-ml/blob/master/ALS_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!ls "/content/drive/My Drive/Colab Notebooks"

ALS-V1.ipynb  data.csv


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



/content/spark-2.2.3-bin-hadoop2.7/python/pyspark/sql/session.py:336: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [0]:
from __future__ import print_function
#!pip install -q pyspark
import sys
if sys.version >= '3':
    long = int

from pyspark.sql import SparkSession

# $example on$
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
# $example off$

if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("ALSExample")\
        .getOrCreate()

# try to load file locally
from google.colab import files
#files.upload()

!ls

#print("well done")

    # $example on$
lines = spark.read.text("data - V2.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(profile_id=int(p[0]), category_id=int(p[1]), score=float(p[2]), category_name=str(p[3])))


# Profile_id and category_id need to be numerical not string
  
ratings = spark.createDataFrame(ratingsRDD)
    
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="profile_id", itemCol="category_id", ratingCol="score", coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="score",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

#Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)


userRecs.show()
movieRecs.show()
userSubsetRecs.show()
movieSubSetRecs.show()


spark.stop()

'data (1).csv'	'data - V2.txt'			 spark-2.3.3-bin-hadoop2.7
'data (2).csv'	 drive				 spark-2.3.3-bin-hadoop2.7.tgz
 data.csv	 sample_data			 spark-warehouse
 data.txt	 spark-2.2.3-bin-hadoop2.7
 data-v1.txt	 spark-2.2.3-bin-hadoop2.7.tgz
well done
Root-mean-square error = 0.6108302068393148


AttributeError: ignored

In [6]:
from __future__ import print_function
#!pip install -q pyspark
import sys
if sys.version >= '3':
    long = int

from pyspark.sql import SparkSession

# $example on$
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
# $example off$

if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("ALSExample")\
        .getOrCreate()

# try to load file locally
from google.colab import files
#files.upload()

!ls

print("well done")

    # $example on$
lines = spark.read.text("data - V2.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(profile_id=int(p[0]), category_id=int(p[1]), score=float(p[2]), category_name=str(p[3])))


# Profile_id and category_id need to be numerical not string
  
ratings = spark.createDataFrame(ratingsRDD)
    
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="profile_id", itemCol="category_id", ratingCol="score", coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="score",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

#Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)


userRecs.show(20, False)
movieRecs.show(20, False)
userSubsetRecs.show(20, False)
movieSubSetRecs.show(20, False)


spark.stop()

'data - V2.txt'   sample_data
well done
Root-mean-square error = 0.26008362160264814
+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|profile_id|recommendations                                                                                                                                                                 |
+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|31        |[[3, 1.2420185], [7, 1.144066], [8, 0.624295], [16, 0.61446124], [2, 0.47210503], [6, 0.46517226], [5, 0.4380749], [17, 0.29845533], [11, 0.29482424], [13, 0.2706228]]         |
|65        |[[3, 3.271118], [7, 3.259867], [16, 1.7018001], [8, 1.3779416], [6, 1.18959], [5, 1.1831832], [4, 0.8303893], [11, 0.8209059], [17, 0.81094694]